In [1]:
# lr, alpha, belta, gamma
from resnet import resnet32
# from cifar import get_cifar10
# import cifar
import utils 
import pencil as pencil_lib

import torchvision.transforms as transforms
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
from scipy.special import softmax

import copy as cp
import os
import pickle
import numpy as np

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
if True: # Define hyper parameters
#     batch_size = 128
    batch_size = 100
    lr = 0.06
    momentum = 0.9
    weight_decay = 10e-4
    n_classes = 10
    pencil_epochs = [70, 130, 120]
    pencil_lrs = [0.06, 0.06, 0.06]
    # pencil_lrs = [0.02, 0.02, 0.02]
    pencil_alpha, pencil_beta, pencil_gamma = 0.1, 0.4, 600
    # pencil_alpha, pencil_beta, pencil_gamma = 0.1, 0.8, 200
    pencil_KL = True
    pencil_K = 10
    checkpoint = 5
    seed = 1
    train_ratio = 0.9
    asym_noise = True
    noise_ratio = 0.2
    temp_path = 'PATH_HERE'
    cifar_path = './data'

if True: # Define dataset and transforms
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    transform_val = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
#     np.random.seed(seed)
#     trainset, valset = get_cifar10(cifar_path, train_ratio=train_ratio, asym=asym_noise, percent=noise_ratio, train=True, download=False, transform_train=transform_train,
#                                                transform_val=transform_val)
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=False, transform=transform_train)
#     print(trainsetori.data[0])
#     trainset = cifar.CIFAR10_train(root='./data', train=True, download=False, transform=transform_train)
#     print(trainset.data[0])
    
    valset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=False, transform=transform_val)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=4)
#     if start_epoch==0: # 
#         print('trainset',len(trainset))
#         trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
#         valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=4)
# ######### Model checkpointing commented out
#         with open(temp_path+"trainloader"+".pkl", "wb") as file:
#             pickle.dump(trainloader,file)
#         with open(temp_path+"valloader"+".pkl", "wb") as file:
#             pickle.dump(valloader,file)
#     else: 
#         with open(temp_path+"trainloader"+".pkl", "rb") as file:
#             trainloader = pickle.load(file)
#         with open(temp_path+"valloader"+".pkl", "rb") as file:
#             valloader = pickle.load(file)


True


In [2]:
if True: # Define Model and optimizer

    # Model
    print("==> creating preact_resnet")
    model = resnet32()
    model = model.to(device)
    print('Total params: %.2fM' % (sum(p.numel() for p in model.parameters()) / 1000000.0))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)


==> creating preact_resnet
Total params: 0.47M


In [3]:
if True: # Define dict for storing results
    results = {'train':None,'val':None}
    metrics = ['acc','auprc','auroc','loss']
    for subset in results.keys(): 
        results[subset] = {metric:[] for metric in metrics}

    results['pencil_labels'], results['labels'] = [], []
    results['pencil_label_drift'], results['pencil_label_correct'] = [], []

    for i in range(len(trainset)): 
        item = trainset.__getitem__(i)
        results['labels'].append(item[1])

if True: # Define PENCIL
    pencil = pencil_lib.PENCIL(torch.tensor(results['labels']), len(trainset), n_classes, pencil_epochs, pencil_lrs, 
                                               pencil_alpha, pencil_beta, pencil_gamma, save_losses=True, use_KL=pencil_KL, K=pencil_K)


In [ ]:
checkpoint = torch.load('weights.tar')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#             initepoch = checkpoint['epoch']
loss = checkpoint['loss']
with open("pencil.pkl", "rb") as file:
    pencil = pickle.load(file)
pencil.alpha=pencil_alpha
pencil.beta=pencil_beta

In [4]:
import time
start_epoch = 0
epochs = 320
for epoch in range(start_epoch,epochs):
    if True: # Train 
        pencil.set_lr(optimizer, epoch)
        model.train()
#         preds, label_list = [], []
#         print('a')

#         if epoch in pencil_epochs: results['pencil_labels'].append(cp.deepcopy(pencil.y_tilde))
        timestart = time.time()
        running_loss = 0.0
        total = 0
        correct = 0
        for batch_idx, (inputs, labels, inds) in enumerate(trainloader):
#             print('e')
            inputs, labels = inputs.to(device), labels.to(device, dtype= torch.int64)
            outputs = model(inputs)
            loss = pencil.get_loss(epoch, outputs, labels, inds)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pencil.update_y_tilde(epoch, inds)
            running_loss += loss.item()
            if batch_idx % 500 == 499:  # print every 500 mini-batches
                    print('[%d, %5d] loss: %.4f' %
                          (epoch, batch_idx, running_loss / 500))
                    running_loss = 0.0
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                    print('Accuracy of the network on the %d tran images: %.3f %%' % (total,
                            100.0 * correct / total))
                    total = 0
                    correct = 0
#             preds.append(outputs.detach())
#             label_list.append(labels.cpu())

#         label_list = np.vstack(torch.cat(label_list).numpy())
#         preds = softmax(np.vstack(torch.cat(preds).cpu().numpy()), axis=1)
#         train_results = utils.get_performance_metrics(num_classes=n_classes,preds=preds,label_list=label_list)
#         for result in train_results.keys():
#             results['train'][result].append(train_results[result])
#         label_drift = round(sum(results['labels']==np.argmax(pencil.y_tilde,axis=1))/len(results['labels']),4)
#         label_true_corr = round(sum(results['true_labels']==np.argmax(pencil.y_tilde,axis=1))/len(results['true_labels']),4)
#         results['pencil_label_drift'].append(label_drift)
#         results['pencil_label_correct'].append(label_true_corr)

    if epoch % 5 == 4:
        correct = 0
        total = 0
        with torch.no_grad():
            for data in valloader:
                images, labels, inds = data
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: %.3f %%' % (
                100.0 * correct / total))                                                          

# ########## Model checkpointing commented out
#     if checkpoint!=None and (epoch%checkpoint==0 or epoch+1 in pencil_epochs): #Save model for this epoch
#         if epoch%checkpoint==0 and epoch>start_epoch+checkpoint: 
#             os.remove(temp_path+'epoch'+str(epoch-checkpoint))
#             os.remove(temp_path+"results"+str(epoch-checkpoint)+".pkl")
#             os.remove(temp_path+"pencil"+str(epoch-checkpoint)+".pkl")
#         torch.save(model.state_dict(), temp_path+'epoch'+str(epoch))
#         with open(temp_path+"results"+str(epoch)+".pkl", "wb") as file:
#             pickle.dump(results,file)
#         with open(temp_path+"pencil"+str(epoch)+".pkl", "wb") as file:
#             pickle.dump(pencil,file)

[0,   499] loss: 1.8937
Accuracy of the network on the 100 tran images: 40.000 %
[1,   499] loss: 1.5633
Accuracy of the network on the 100 tran images: 51.000 %
[2,   499] loss: 1.3678
Accuracy of the network on the 100 tran images: 64.000 %
[3,   499] loss: 1.2372
Accuracy of the network on the 100 tran images: 56.000 %
[4,   499] loss: 1.1551
Accuracy of the network on the 100 tran images: 62.000 %
Accuracy of the network on the 10000 test images: 68.900 %
[5,   499] loss: 1.1069
Accuracy of the network on the 100 tran images: 77.000 %
[6,   499] loss: 1.0838
Accuracy of the network on the 100 tran images: 70.000 %
[7,   499] loss: 1.0597
Accuracy of the network on the 100 tran images: 75.000 %
[8,   499] loss: 1.0442
Accuracy of the network on the 100 tran images: 71.000 %
[9,   499] loss: 1.0307
Accuracy of the network on the 100 tran images: 69.000 %
Accuracy of the network on the 10000 test images: 73.850 %
[10,   499] loss: 1.0183
Accuracy of the network on the 100 tran images:

C:\Users\Administrator\anaconda3\lib\site-packages\torch\nn\functional.py:2610: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


[70,   499] loss: 0.3904
Accuracy of the network on the 100 tran images: 76.000 %
[71,   499] loss: 0.3886
Accuracy of the network on the 100 tran images: 73.000 %
[72,   499] loss: 0.3848
Accuracy of the network on the 100 tran images: 77.000 %
[73,   499] loss: 0.3746
Accuracy of the network on the 100 tran images: 71.000 %
[74,   499] loss: 0.3588
Accuracy of the network on the 100 tran images: 80.000 %
Accuracy of the network on the 10000 test images: 76.350 %
[75,   499] loss: 0.3399
Accuracy of the network on the 100 tran images: 73.000 %
[76,   499] loss: 0.3235
Accuracy of the network on the 100 tran images: 71.000 %
[77,   499] loss: 0.3069
Accuracy of the network on the 100 tran images: 73.000 %
[78,   499] loss: 0.2888
Accuracy of the network on the 100 tran images: 76.000 %
[79,   499] loss: 0.2789
Accuracy of the network on the 100 tran images: 71.000 %
Accuracy of the network on the 10000 test images: 76.240 %
[80,   499] loss: 0.2668
Accuracy of the network on the 100 tr

[157,   499] loss: 0.2129
Accuracy of the network on the 100 tran images: 76.000 %
[158,   499] loss: 0.2120
Accuracy of the network on the 100 tran images: 68.000 %
[159,   499] loss: 0.2133
Accuracy of the network on the 100 tran images: 80.000 %
Accuracy of the network on the 10000 test images: 74.930 %
[160,   499] loss: 0.2112
Accuracy of the network on the 100 tran images: 71.000 %
[161,   499] loss: 0.2128
Accuracy of the network on the 100 tran images: 73.000 %
[162,   499] loss: 0.2120
Accuracy of the network on the 100 tran images: 68.000 %
[163,   499] loss: 0.2129
Accuracy of the network on the 100 tran images: 66.000 %
[164,   499] loss: 0.2115
Accuracy of the network on the 100 tran images: 70.000 %
Accuracy of the network on the 10000 test images: 73.240 %
[165,   499] loss: 0.2103
Accuracy of the network on the 100 tran images: 73.000 %
[166,   499] loss: 0.2120
Accuracy of the network on the 100 tran images: 77.000 %
[167,   499] loss: 0.2120
Accuracy of the network on

[244,   499] loss: 0.0111
Accuracy of the network on the 100 tran images: 81.000 %
Accuracy of the network on the 10000 test images: 81.120 %
[245,   499] loss: 0.0108
Accuracy of the network on the 100 tran images: 79.000 %
[246,   499] loss: 0.0107
Accuracy of the network on the 100 tran images: 74.000 %
[247,   499] loss: 0.0105
Accuracy of the network on the 100 tran images: 71.000 %
[248,   499] loss: 0.0106
Accuracy of the network on the 100 tran images: 82.000 %
[249,   499] loss: 0.0103
Accuracy of the network on the 100 tran images: 77.000 %
Accuracy of the network on the 10000 test images: 81.380 %
[250,   499] loss: 0.0103
Accuracy of the network on the 100 tran images: 74.000 %
[251,   499] loss: 0.0101
Accuracy of the network on the 100 tran images: 73.000 %
[252,   499] loss: 0.0100
Accuracy of the network on the 100 tran images: 72.000 %
[253,   499] loss: 0.0101
Accuracy of the network on the 100 tran images: 71.000 %
[254,   499] loss: 0.0100
Accuracy of the network on

In [5]:
torch.save({'epoch':epoch,
                        'model_state_dict':model.state_dict(),
                        'optimizer_state_dict':optimizer.state_dict(),
                        'loss':loss
                        },'weights.tar')
with open("pencil.pkl", "wb") as file:
    pickle.dump(pencil,file)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import os
import numpy as np

def test(model,device):
        correct = 0
        total = 0
        with torch.no_grad():
            for data in valloader:
                images, labels, inds = data
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: %.3f %%' % (
                100.0 * correct / total))

In [7]:
test(model,device)

Accuracy of the network on the 10000 test images: 83.200 %
